In [1]:
import geopandas as gpd

In [ ]:
validation_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb"
validation_layer_name ="BLM_enriched2023_20240918"
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\BLM_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"BLM_enriched_{datetime.today().strftime('%Y%m%d')}"
